In [ ]:
#Packages
import os
import pandas as pd
import numpy as np
import xgboost
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score, roc_curve
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
import matplotlib.pyplot as plt
import seaborn as sns
import shap

In [ ]:
#Get Data
csv_dir = '../../SEC Trackman Data/'
all_files = [os.path.join(csv_dir, f) for f in os.listdir(csv_dir) if f.endswith('.csv')]

df_list = []

for file in all_files:
    df = pd.read_csv(file)
    df_list.append(df)

all_pitches = pd.concat(df_list, ignore_index=True)

print(all_pitches.shape)
print(all_pitches.columns)

In [ ]:
#Get all curveballs
bbc = all_pitches[all_pitches['TaggedPitchType'].isin(['Curveball'])]
print(bbc.shape)

In [ ]:
bbc['whiff'] = np.where(bbc['PitchCall']=='StrikeSwinging',1,0)

In [ ]:
preds = ['RelSpeed','VertRelAngle','HorzRelAngle','SpinRate','SpinAxis','RelHeight','RelSide','Extension','VertBreak','InducedVertBreak','HorzBreak','ZoneSpeed','VertApprAngle','HorzApprAngle','ZoneTime','pfxx','pfxz','x0','y0','z0','vx0','vy0','vz0','ax0','ay0','az0','SpeedDrop','PitchTrajectoryXc1','PitchTrajectoryXc2','PitchTrajectoryYc0','PitchTrajectoryYc1','PitchTrajectoryYc2','PitchTrajectoryZc0','PitchTrajectoryZc1','PitchTrajectoryZc2']

In [ ]:
bbc_X = bbc[preds]
bbc_y = bbc['whiff']

In [ ]:
#Train Model
X_train, X_test, y_train, y_test = train_test_split(bbc_X, bbc_y, test_size=.33, random_state=25)

model = XGBClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
#Auburn Subset
aub_bbc = bbc[bbc['PitcherTeam']=='AUB_TIG']

In [ ]:
# Get unique values
unique_values = bbc['Pitcher'].unique()
aub_unique_values = aub_bbc['Pitcher'].unique()

#Print list of Auburn pitchers for selected split
print(aub_unique_values)

# Create a dictionary to store subsets
all_subsets = {value: bbc[bbc['Pitcher'] == value] for value in unique_values}

aub_subsets = {value: bbc[bbc['Pitcher'] == value] for value in aub_unique_values}

In [ ]:
def aub_bbc_stuff(name):
    
    preds = ['RelSpeed','VertRelAngle','HorzRelAngle','SpinRate','SpinAxis','RelHeight','RelSide','Extension','VertBreak','InducedVertBreak','HorzBreak','ZoneSpeed','VertApprAngle','HorzApprAngle','ZoneTime','pfxx','pfxz','x0','y0','z0','vx0','vy0','vz0','ax0','ay0','az0','SpeedDrop','PitchTrajectoryXc1','PitchTrajectoryXc2','PitchTrajectoryYc0','PitchTrajectoryYc1','PitchTrajectoryYc2','PitchTrajectoryZc0','PitchTrajectoryZc1','PitchTrajectoryZc2']
    
    all_preds = model.predict_proba(bbc_X)[:,1]
    mean_pred = np.mean(all_preds)

    pitcher = aub_subsets[name]
    pitcher_x = pitcher[preds]

    whiff_preds = model.predict_proba(pitcher_x)[:,1]

    stuff = whiff_preds/mean_pred
    
    print(f"{name} Curveball stuff+: {np.mean(stuff * 100)}")

In [ ]:
#Stuff+ for selected Auburn pitchers Curveball
for pitcher in aub_unique_values:
    aub_bbc_stuff(pitcher)

Now, let's do splits

In [ ]:
#Subset curveballs into platoon splits
bbc_rr = bbc[(bbc['PitcherThrows'] == 'Right') & (bbc['BatterSide'] == 'Right')]
bbc_rl = bbc[(bbc['PitcherThrows'] == 'Right') & (bbc['BatterSide'] == 'Left')]
bbc_lr = bbc[(bbc['PitcherThrows'] == 'Left') & (bbc['BatterSide'] == 'Right')]
bbc_ll = bbc[(bbc['PitcherThrows'] == 'Left') & (bbc['BatterSide'] == 'Left')]

print(bbc_rr.shape)
print(bbc_rl.shape)
print(bbc_lr.shape)
print(bbc_ll.shape)

In [ ]:
#Whiffs
bbc_rr['whiff'] = np.where(bbc_rr['PitchCall']=='StrikeSwinging',1,0)
bbc_rl['whiff'] = np.where(bbc_rl['PitchCall']=='StrikeSwinging',1,0)
bbc_lr['whiff'] = np.where(bbc_lr['PitchCall']=='StrikeSwinging',1,0)
bbc_ll['whiff'] = np.where(bbc_ll['PitchCall']=='StrikeSwinging',1,0)

In [ ]:
#Predictor Variables
preds = ['RelSpeed','VertRelAngle','HorzRelAngle','SpinRate','SpinAxis','RelHeight','RelSide','Extension','VertBreak','InducedVertBreak','HorzBreak','ZoneSpeed','VertApprAngle','HorzApprAngle','ZoneTime','pfxx','pfxz','x0','y0','z0','vx0','vy0','vz0','ax0','ay0','az0','SpeedDrop','PitchTrajectoryXc1','PitchTrajectoryXc2','PitchTrajectoryYc0','PitchTrajectoryYc1','PitchTrajectoryYc2','PitchTrajectoryZc0','PitchTrajectoryZc1','PitchTrajectoryZc2']

Righty vs. Righty Split

In [ ]:
bbc_rr_X = bbc_rr[preds]
bbc_rr_y = bbc_rr['whiff']

In [ ]:
#Train Model: RHP vs RHB
X_train, X_test, y_train, y_test = train_test_split(bbc_rr_X, bbc_rr_y, test_size=.33, random_state=25)

model = XGBClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
y_pred_prob = model.predict_proba(X_test)[:,1]
roc_auc = roc_auc_score(y_test, predictions, multi_class='ovr')

print(f'ROC AUC: {roc_auc:.4f}')

# Plotting ROC Curve for one class (e.g., class 0)
fpr, tpr, _ = roc_curve(y_test == 1, y_pred_prob)
plt.plot(fpr, tpr, label='Class 0 ROC curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc='best')
plt.show()

In [ ]:
#Auburn Subset
aub_bbc_rr = bbc_rr[bbc_rr['PitcherTeam']=='AUB_TIG']
print(aub_bbc_rr.shape)

In [ ]:
# Get unique values
unique_values = bbc_rr['Pitcher'].unique()
aub_unique_values = aub_bbc_rr['Pitcher'].unique()


#Print list of Auburn pitchers for selected split
print(aub_unique_values)

# Create a dictionary to store subsets
subsets = {value: bbc_rr[bbc_rr['Pitcher'] == value] for value in unique_values}
aub_subsets = {value: bbc_rr[bbc_rr['Pitcher'] == value] for value in aub_unique_values}

In [ ]:
def aub_rr_bbc_stuff(name):
    
    preds = ['RelSpeed','VertRelAngle','HorzRelAngle','SpinRate','SpinAxis','RelHeight','RelSide','Extension','VertBreak','InducedVertBreak','HorzBreak','ZoneSpeed','VertApprAngle','HorzApprAngle','ZoneTime','pfxx','pfxz','x0','y0','z0','vx0','vy0','vz0','ax0','ay0','az0','SpeedDrop','PitchTrajectoryXc1','PitchTrajectoryXc2','PitchTrajectoryYc0','PitchTrajectoryYc1','PitchTrajectoryYc2','PitchTrajectoryZc0','PitchTrajectoryZc1','PitchTrajectoryZc2']
    
    all_preds = model.predict_proba(bbc_rr_X)[:,1]
    mean_pred = np.mean(all_preds)

    pitcher = aub_subsets[name]
    pitcher_x = pitcher[preds]

    whiff_preds = model.predict_proba(pitcher_x)[:,1]

    stuff = whiff_preds/mean_pred
    
    print(f"{name} curveball rr stuff+: {np.mean(stuff * 100)}")

    


In [ ]:
#Stuff+ for selected Auburn pitchers Curveball in RHP v RHB split
for pitcher in aub_unique_values:
    aub_rr_bbc_stuff(pitcher)

In [ ]:
aub_subsets['Murphy, Hayden']

Righty vs. Lefty Split

In [ ]:
bbc_rl_X = bbc_rl[preds]
bbc_rl_y = bbc_rl['whiff']

In [ ]:
#Train Model: RHP vs LHB
X_train, X_test, y_train, y_test = train_test_split(bbc_rl_X, bbc_rl_y, test_size=.33, random_state=25)

model = XGBClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
#Auburn Subset
aub_bbc_rl = bbc_rl[bbc_rl['PitcherTeam']=='AUB_TIG']
print(aub_bbc_rl.shape)

In [ ]:
# Get unique values
unique_values = bbc_rl['Pitcher'].unique()
aub_unique_values = aub_bbc_rl['Pitcher'].unique()

#Print list of Auburn pitchers for selected split
print(aub_unique_values)

# Create a dictionary to store subsets
subsets = {value: bbc_rl[bbc_rl['Pitcher'] == value] for value in unique_values}
aub_subsets = {value: bbc_rl[bbc_rl['Pitcher'] == value] for value in aub_unique_values}

In [ ]:
def aub_rl_bbc_stuff(name):
    
    preds = ['RelSpeed','VertRelAngle','HorzRelAngle','SpinRate','SpinAxis','RelHeight','RelSide','Extension','VertBreak','InducedVertBreak','HorzBreak','ZoneSpeed','VertApprAngle','HorzApprAngle','ZoneTime','pfxx','pfxz','x0','y0','z0','vx0','vy0','vz0','ax0','ay0','az0','SpeedDrop','PitchTrajectoryXc1','PitchTrajectoryXc2','PitchTrajectoryYc0','PitchTrajectoryYc1','PitchTrajectoryYc2','PitchTrajectoryZc0','PitchTrajectoryZc1','PitchTrajectoryZc2']
    
    all_preds = model.predict_proba(bbc_rl_X)[:,1]
    mean_pred = np.mean(all_preds)

    pitcher = aub_subsets[name]
    pitcher_x = pitcher[preds]

    whiff_preds = model.predict_proba(pitcher_x)[:,1]

    stuff = whiff_preds/mean_pred
    
    print(f"{name} curveball rl stuff+: {np.mean(stuff * 100)}")


In [ ]:
#Stuff+ for selected Auburn pitchers Curveball in RHP v LHB split
for pitcher in aub_unique_values:
    aub_rl_bbc_stuff(pitcher)

Lefty vs. Righty Split

In [ ]:
bbc_lr_X = bbc_lr[preds]
bbc_lr_y = bbc_lr['whiff']

In [ ]:
#Train Model: LHP vs RHB
X_train, X_test, y_train, y_test = train_test_split(bbc_lr_X, bbc_lr_y, test_size=.33, random_state=25)

model = XGBClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
#Auburn Subset
aub_bbc_lr = bbc_lr[bbc_lr['PitcherTeam']=='AUB_TIG']
print(aub_bbc_lr.shape)

In [ ]:
# Get unique values
unique_values = bbc_lr['Pitcher'].unique()
aub_unique_values = aub_bbc_lr['Pitcher'].unique()

#Print list of Auburn pitchers for selected split
print(aub_unique_values)

# Create a dictionary to store subsets
subsets = {value: bbc_lr[bbc_lr['Pitcher'] == value] for value in unique_values}
aub_subsets = {value: bbc_lr[bbc_lr['Pitcher'] == value] for value in aub_unique_values}



In [ ]:
def aub_lr_bbc_stuff(name):
    
    preds = ['RelSpeed','VertRelAngle','HorzRelAngle','SpinRate','SpinAxis','RelHeight','RelSide','Extension','VertBreak','InducedVertBreak','HorzBreak','ZoneSpeed','VertApprAngle','HorzApprAngle','ZoneTime','pfxx','pfxz','x0','y0','z0','vx0','vy0','vz0','ax0','ay0','az0','SpeedDrop','PitchTrajectoryXc1','PitchTrajectoryXc2','PitchTrajectoryYc0','PitchTrajectoryYc1','PitchTrajectoryYc2','PitchTrajectoryZc0','PitchTrajectoryZc1','PitchTrajectoryZc2']
    
    all_preds = model.predict_proba(bbc_lr_X)[:,1]
    mean_pred = np.mean(all_preds)

    pitcher = aub_subsets[name]
    pitcher_x = pitcher[preds]

    whiff_preds = model.predict_proba(pitcher_x)[:,1]

    stuff = whiff_preds/mean_pred
    
    print(f"{name} curveball lr stuff+: {np.mean(stuff * 100)}")


In [ ]:
#Stuff+ for selected Auburn pitchers Curveball in LHP v RHB split
for pitcher in aub_unique_values:
    aub_lr_bbc_stuff(pitcher)

Lefty vs. Lefty Split

In [ ]:
bbc_ll_X = bbc_ll[preds]
bbc_ll_y = bbc_ll['whiff']

In [ ]:
#Train Model: LHP vs LHB
X_train, X_test, y_train, y_test = train_test_split(bbc_ll_X, bbc_ll_y, test_size=.33, random_state=25)

model = XGBClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
#Auburn Subset
aub_bbc_ll = bbc_ll[bbc_ll['PitcherTeam']=='AUB_TIG']
print(aub_bbc_ll.shape)

In [ ]:
# Get unique values
unique_values = bbc_ll['Pitcher'].unique()
aub_unique_values = aub_bbc_ll['Pitcher'].unique()

#Print list of Auburn pitchers for selected split
print(aub_unique_values)

# Create a dictionary to store subsets
subsets = {value: bbc_ll[bbc_ll['Pitcher'] == value] for value in unique_values}
aub_subsets = {value: bbc_ll[bbc_ll['Pitcher'] == value] for value in aub_unique_values}



In [ ]:
def aub_ll_bbc_stuff(name):
    
    preds = ['RelSpeed','VertRelAngle','HorzRelAngle','SpinRate','SpinAxis','RelHeight','RelSide','Extension','VertBreak','InducedVertBreak','HorzBreak','ZoneSpeed','VertApprAngle','HorzApprAngle','ZoneTime','pfxx','pfxz','x0','y0','z0','vx0','vy0','vz0','ax0','ay0','az0','SpeedDrop','PitchTrajectoryXc1','PitchTrajectoryXc2','PitchTrajectoryYc0','PitchTrajectoryYc1','PitchTrajectoryYc2','PitchTrajectoryZc0','PitchTrajectoryZc1','PitchTrajectoryZc2']
    
    all_preds = model.predict_proba(bbc_ll_X)[:,1]
    mean_pred = np.mean(all_preds)

    pitcher = aub_subsets[name]
    pitcher_x = pitcher[preds]

    whiff_preds = model.predict_proba(pitcher_x)[:,1]

    stuff = whiff_preds/mean_pred
    
    print(f"{name} curveball ll stuff+: {np.mean(stuff * 100)}")


In [ ]:
#Stuff+ for selected Auburn pitchers Curveball in LHP v LHB split
for pitcher in aub_unique_values:
    aub_ll_bbc_stuff(pitcher)